<a href="https://www.kaggle.com/code/ishansrivastava1308/commit-notebook-of-spotify-data-scraper?scriptVersionId=186730796" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install pytube --upgrade
!pip install spotipy
!pip install ytmusicapi
# !pip install yt-dlp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 2.0 MB/s eta 0:00:00


In [2]:
from ytmusicapi import YTMusic
import ytmusicapi
from pprint import pprint
import os
from pytube import YouTube
import urllib.request
import requests
import string
from spotipy.client import Spotify
from spotipy import SpotifyOAuth
from spotipy.oauth2 import SpotifyOauthError, SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import concurrent.futures
import time
import threading
from pydub import AudioSegment
from IPython.display import Audio
from dotenv import load_dotenv
import multiprocessing
from tqdm import tqdm
from IPython.display import Audio
# import yt_dlp

In [3]:
import pytube.innertube as pti

In [4]:
pti._token_file = '/kaggle/working/tokens.json'
pti._cache_dir = '/kaggle/working'

# Making Kaggle Dataset

In [5]:
import os
import json

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

os.environ['KAGGLE_USERNAME'] = secrets.get_secret("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = secrets.get_secret("KAGGLE_KEY")

In [6]:
meta = dict(
    id="ishansrivastava1308/spotify-dataset",
    title="Spotify Dataset",
    isPrivate=True,
    licenses=[dict(name="other")]
)
os.makedirs('/kaggle/working/dataset', exist_ok = True)
with open(os.path.join('dataset','dataset-metadata.json'), 'w') as f:
    json.dump(meta, f)

In [7]:
# !kaggle datasets create -p /kaggle/working/dataset --dir-mode tar

In [8]:
# !kaggle datasets version -m"Added more songs and data" -p /kaggle/working/dataset --dir-mode tar

# Scrapper Class Definition

In [9]:
df_columns = ['track_name', 'track_id', 'track_number', 'disc_number', 'duration_ms',
       'explicit', 'popularity', 'preview_url', 'isrc', 'album_name',
       'album_id', 'album_type', 'album_total_tracks', 'album_release_date',
       'album_release_date_precision', 'album_images', 'popular_artist',
       'popular_artist_id', 'artist_names', 'artist_ids', 'combined_genres',
       'artist_popularity', 'artist_followers', 'external_url', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence',
       'lyrics', 'audio_file_path', 'video_id']

In [10]:
class YTScraper:
    def __init__(
        self,
        token_file_path="oauth.json",
    ):
        self.yt_music = YTMusic(token_file_path)
        YouTube(
            "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
            use_oauth=True,
            allow_oauth_cache=True,
        ).streams.filter(only_audio=True).first().download()

    def get_search_results(self, search_str):
        return self.yt_music.search(search_str, filter="songs")

    def get_lyrics(self, video_id):
        video = self.yt_music.get_watch_playlist(
            videoId=video_id,
        )
        lyrics_id = video["lyrics"]
        lyrics = None
        if lyrics_id:
            lyrics = self.yt_music.get_lyrics(lyrics_id)
        return lyrics

    #   def download_song_by_video_id(
    #         self,
    #         video_id,
    #         output_path=None,
    #         audio_format = 'mp3'
    #   ):
    #         if output_path is not None:
    #             os.makedirs(output_path,exist_ok=True)
    #         else:
    #             output_path = ""
    #         ydl_opts = {
    #             'format': 'm4a/bestaudio/best',
    #             # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    #             'postprocessors': [{  # Extract audio using ffmpeg
    #                 'key': 'FFmpegExtractAudio',
    #                 'preferredcodec': 'mp3',
    #             }]
    #         }

    #         with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    #             error_code = ydl.download()

    def download_song_by_video_id(
        self,
        video_id,
        output_path=None,
    ):
        if output_path is not None:
            os.makedirs(output_path, exist_ok=True)
        else:
            output_path = ""
        yt = YouTube(f"https://youtube.com/watch?v={video_id}")
        temp_file = os.path.join(output_path, f"{video_id}.mp4")
        yt.streams.filter(only_audio=True).first().download(
            output_path=output_path, filename=f"{video_id}.mp4"
        )
        if os.path.exists(temp_file):
            #         print(f"Downloaded successfully: {temp_file}")
            audio = AudioSegment.from_file(temp_file, format="mp4")
            audio = audio.set_frame_rate(16000)
            audio = audio.set_channels(1)
            wav_path = temp_file[:-4] + ".mp3"
            audio.export(wav_path, format="mp3")
            os.remove(temp_file)

        return os.path.join(output_path, f"{video_id}.mp3")

    def threaded_download_song_by_video_id_list(
        self, video_id_list, output_path="songs", max_workers=50
    ):
        data = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # List of tasks to be executed by the thread pool
            output_path_args = [output_path] * len(video_id_list)
            tasks = [
                executor.submit(self.download_song_by_video_id, *args)
                for args in zip(video_id_list, output_path_args)
            ]
            success = 0
            fails = 0

            with tqdm(total=len(video_id_list)) as pbar:
                for future in concurrent.futures.as_completed(tasks):
                    try:
                        data.append(future.result())
                        pbar.update(1)
                        success += 1
                    except Exception as exc:
                        print(f"Task generated an exception: {exc}")
                        fails += 1
            print({"success": success, "fail": fails})
        return np.array(data)

    def scrap_by_search(
        self,
        search_str,
        download_params={
            "include_audio": True,
            "include_lyrics": True,
            "output_path": "songs",
        },
    ):
        include_lyrics = download_params["include_lyrics"]
        include_audio = download_params["include_audio"]
        output_path = download_params["output_path"]

        search_result = self.get_search_results(search_str=search_str)
        video_id = None
        for item in search_result:
            if item["resultType"] in ["song"] and item["category"] == "Songs":
                video_id = item["videoId"]
                break
        lyrics = {"lyrics": None, "source": None}
        download_path = None
        if video_id:
            if include_lyrics:
                lyrics = self.get_lyrics(video_id=video_id)
                if not lyrics:
                    lyrics = {"lyrics": None, "source": None}
            if include_audio:
                download_path = self.download_song_by_video_id(
                    video_id=video_id, output_path=output_path
                )

        return {
            "lyrics": lyrics["lyrics"],
            "audio_file_path": download_path,
            "video_id": video_id,
        }

In [11]:
class SpotifyScraper:
    def __init__(
        self,
        client_id,
        client_secret,
#         redirect_uri,
        scope = "user-library-read playlist-read-private playlist-read-collaborative"
        
    ):
        self.df = pd.DataFrame(columns = df_columns)
        self.yt_scraper = YTScraper()
        self._credentials = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
        self.sp = Spotify(auth = self.get_access_token())
        self.download_params = {
            'include_audio' : True,
            'include_lyrics' : True,
            'output_path' : 'songs'
        }
            
    def get_access_token(self):
        # Get access token
        access_token = self._credentials.get_access_token(as_dict = False)
        if not access_token:
            raise Exception("Access Token Not Found")
        return access_token

    def get_auth_header(self):
        return {"Authorization": "Bearer " + self.get_access_token()}
    # Function to fetch user playlists
    
    def get_user_playlists(self):
        print("Retrieving user playlists...")
        headers = self.get_auth_header()
        response = requests.get("https://api.spotify.com/v1/me/playlists", headers=headers)
        response_json = response.json()
        # for item in response_json["items"]:
        #     playlists[item["name"]] = item["id"]
        print("Playlists retrieved successfully.")
        return response_json
    
    @staticmethod
    def sanitize_filename(filename):
        valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
        return ''.join(c for c in filename if c in valid_chars)

    def get_track_info_by_id(self, track_id):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/tracks/{track_id}", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def get_playlist_info_by_id(self, playlist_id):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def get_several_track_info_by_id(self, track_ids_str):
        for i in range(3):
            headers = self.get_auth_header()
            response = requests.get(f"https://api.spotify.com/v1/tracks?ids={track_ids_str}", headers=headers)
            if not response.ok:
                retry_after = response.headers.get('Retry-After')
                if (retry_after):
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(int(retry_after) + 1)
                else:
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(10)
            else:
                response_json = response.json()
                return response_json
    
    def get_several_artist_info_by_id(self, artist_ids_str):
        for i in range(3):
            headers = self.get_auth_header()
            response = requests.get(f"https://api.spotify.com/v1/artists?ids={artist_ids_str}", headers=headers)
            if not response.ok:
                retry_after = response.headers.get('Retry-After')
                if (retry_after):
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(int(retry_after) + 1)
                else:
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(10)
            else:
                response_json = response.json()
                return response_json
    
    def get_several_audio_feature_by_id(self, track_ids_str):
        for i in range(3):
            headers = self.get_auth_header()
            response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={track_ids_str}", headers=headers)
            if not response.ok:
                retry_after = response.headers.get('Retry-After')
                if (retry_after):
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(int(retry_after) + 1)
                else:
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(10)
            else:
                response_json = response.json()
                return response_json
    
    def destructure_artist_data(self,artists_list):
        artist_infos = []
        artist_genres = []
        artist_popularities = []
        artist_followers = []
        for artist_obj in artists_list:
            artist_infos.append({
                "name": artist_obj["name"],
                "id": artist_obj["id"],
                "genres": artist_obj.get("genres", []),
                "popularity": artist_obj["popularity"],
                "followers": artist_obj["followers"]["total"]
            })
            artist_genres.extend(artist_obj.get("genres", []))
            artist_popularities.append(artist_obj["popularity"])
            artist_followers.append(artist_obj["followers"]["total"])
        return {
            'artist_infos' : artist_infos,
            'artist_genres' : artist_genres,
            'artist_popularities' : artist_popularities,
            'artist_followers' : artist_followers,
        }
        
    def get_playlist_by_id(self,playlist_id : str):
        return self.sp.playlist_tracks(playlist_id = playlist_id)    
    
    def construct_track_info_dict(self,track_json = None, audio_features_json = None, artist_json = None): 
        
        most_popular_artist = max(artist_json['artist_infos'], key=lambda x: x["popularity"])
        
        search_str = f"{track_json['name']} {most_popular_artist['name']}"
        
        lyrics_audio_data = self.yt_scraper.scrap_by_search(search_str=search_str, download_params = self.download_params)
        
        track_info = {
            "track_name": track_json["name"],
            "track_id": track_json["id"],
            "track_number": track_json["track_number"],
            "disc_number": track_json["disc_number"],
            "duration_ms": track_json["duration_ms"],
            "explicit": track_json["explicit"],
            "popularity": track_json["popularity"],
            "preview_url": track_json["preview_url"],
            "isrc": track_json["external_ids"].get("isrc"),
            "album_name": track_json["album"]["name"],
            "album_id": track_json["album"]["id"],
            "album_type": track_json["album"]["album_type"],
            "album_total_tracks": track_json["album"]["total_tracks"],
            "album_release_date": track_json["album"]["release_date"],
            "album_release_date_precision": track_json["album"]["release_date_precision"],
            "album_images": track_json["album"]["images"],
            "popular_artist": most_popular_artist["name"],
            "popular_artist_id": most_popular_artist["id"],
            "artist_names": [artist["name"] for artist in track_json["artists"]],
            "artist_ids": [artist["id"] for artist in track_json["artists"]],
            "combined_genres": list(set(artist_json['artist_genres'])),
            "artist_popularity": most_popular_artist["popularity"],
            "artist_followers": most_popular_artist["followers"],
            "external_url": track_json["external_urls"]["spotify"],
            "acousticness": audio_features_json["acousticness"],
            "danceability": audio_features_json["danceability"],
            "energy": audio_features_json["energy"],
            "instrumentalness": audio_features_json["instrumentalness"],
            "key": audio_features_json["key"],
            "liveness": audio_features_json["liveness"],
            "loudness": audio_features_json["loudness"],
            "mode": audio_features_json["mode"],
            "speechiness": audio_features_json["speechiness"],
            "tempo": audio_features_json["tempo"],
            "time_signature": audio_features_json["time_signature"],
            "valence": audio_features_json["valence"],
        }
        track_info.update(lyrics_audio_data)
        return track_info
        
    def _get_single_threaded_data(self,track_obj, audio_feat_obj):
        artist_ids_list = [artist['id'] for artist in track_obj["artists"]]
        artists_id_str = ','.join(artist_ids_list[:50])
        artists_data = self.get_several_artist_info_by_id(artists_id_str)["artists"]
        destructured_artist_data = self.destructure_artist_data(artists_list=artists_data)  
        return self.construct_track_info_dict(track_obj, audio_feat_obj, destructured_artist_data)
    
    def _get_multi_threaded_data(self,tracks_json,audio_features_json, max_workers):
        data = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # List of tasks to be executed by the thread pool
            tasks = [
                executor.submit(self._get_single_threaded_data , *args) for args in zip(tracks_json, audio_features_json)
            ]
            success = 0
            fails = 0
            
            with tqdm(total=len(tracks_json)) as pbar:
                for future in concurrent.futures.as_completed(tasks):
                    try:
                        data.append(future.result())
                        pbar.update(1)
                        success+=1
                    except Exception as exc:
                        print(f'Task generated an exception: {exc}') 
                        fails+=1
            print({'success' : success, 'fail' : fails})
                
        return data
    
    def _get_multiprocessed_data(self,tracks_json,audio_features_json, max_workers):
        data = []
        with multiprocessing.Pool(processes=max_workers) as pool:
            # List of tasks to be executed by the thread pool
            tasks = [
                pool.apply_async(self._get_single_threaded_data , *args) for args in zip(tracks_json, audio_features_json)
            ]
            
            for task in tasks:
                # try:
                    result = task.get()
                    print(result)
                    data.append(task.get())
                # except Exception as exc:
                    # print(f'Task generated an exception: {exc}') 
        return data
    
    
    def get_several_track_data(self, track_ids_list : list[str], parallelization = 0, max_workers = 50, include_audio = True, include_lyrics = True, output_path = "songs"):
        data = []
        track_ids_list = list(set(track_ids_list))
        self.download_params['include_audio'] = include_audio
        self.download_params['include_lyrics'] = include_lyrics
        self.download_params['output_path'] = output_path
        with tqdm(total = len(track_ids_list)) as pbar:
            for idx in range(0,len(track_ids_list),50):
                track_id_str = ",".join(track_ids_list[idx:idx+50])
                tracks_json = self.get_several_track_info_by_id(track_id_str)["tracks"]
                audio_features_json = self.get_several_audio_feature_by_id(track_ids_str=track_id_str)["audio_features"]
                if parallelization == 1:
                    data.extend(self._get_multi_threaded_data(tracks_json,audio_features_json,max_workers))
                elif False and parallelization == 2:
                    data.extend(self._get_multiprocessed_data(tracks_json,audio_features_json,max_workers))
                else:
                    temp_data = []
                    for track_obj, audio_feat_obj in zip(tracks_json, audio_features_json):
                        temp_data.append(self._get_single_threaded_data(track_obj, audio_feat_obj))
                        pbar.update(1)
                    data.extend(temp_data)
                    self.df = pd.concat([self.df,pd.DataFrame(temp_data)],ignore_index = True)
        return data
    
    def get_track_ids_from_playlist(self,*args,**kwargs):
        playlist_data = self.sp.playlist_tracks(*args,**kwargs)
        return [item['track']['id'] for item in playlist_data['items']]
    
    def get_track_ids_from_album(self,*args,**kwargs):
        album_data = self.sp.album_tracks(*args,**kwargs)
        return [item['id'] for item in album_data['items']]
    
    
      

## Loading the Environment and Authentication

### Loading in Kaggle Environment

In [12]:
import os
import json

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

os.environ['KAGGLE_USERNAME'] = secrets.get_secret("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = secrets.get_secret("KAGGLE_KEY")
client_id = secrets.get_secret("SPOTIFY_CLIENT_ID")
client_secret = secrets.get_secret("SPOTIFY_CLIENT_SECRET")
# redirect_uri = secrets.get_secret("REDIRECT_URI")
yt_music_oauth_metadata = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "Content-Type": "application/json",
    "X-Goog-AuthUser": "0",
    "x-origin": "https://music.youtube.com",
    "Cookie" : secrets.get_secret("YT_MUSIC_COOKIE")
}

with open(os.path.join('/kaggle/working/','oauth.json'), 'w') as f:
    json.dump(yt_music_oauth_metadata, f)
    
with open('/kaggle/working/tokens.json','w') as f:
    json.dump(eval(secrets.get_secret('YT_TOKEN')), f)

## Scrapper Instance

In [13]:
scrapper = SpotifyScraper(
  client_id=client_id,
  client_secret=client_secret,
#   redirect_uri=redirect_uri,
)

In [14]:
# with open('/kaggle/working/tokens.json','r') as f:
#     dic = json.load(f)
    
# print(dic)

In [15]:
# loading songs json
with open('/kaggle/input/spotify-metadata-audio-dataset-001/songs.json','r') as f:
    songs_list = json.load(f)

In [16]:
track_ids = songs_list[0:8400] 
# track_ids = songs_list[8400:16800] 
# track_ids = songs_list[16800:]

In [17]:
try:
    track_data = scrapper.get_several_track_data(
        track_ids,
        parallelization=0,
        output_path = 'dataset/songs', 
        include_audio = False, 
        include_lyrics = True,
        max_workers = 1,
    )
    track_data_df = pd.DataFrame(track_data)
    track_data_df.to_pickle('dataset/total_data.pkl')
    scrapper.df.to_pickle('dataset/extracted_data.pkl')
except :
    scrapper.df.to_pickle('dataset/extracted_data.pkl')
    !kaggle datasets version -m "Ishan 0 to 8400 metadata done" -p /kaggle/working/dataset --dir-mode tar

  1%|          | 50/8400 [00:43<1:54:04,  1.22it/s]/tmp/ipykernel_18/581792785.py:254: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df,pd.DataFrame(temp_data)],ignore_index = True)
  3%|▎         | 221/8400 [03:18<2:02:35,  1.11it/s]


Starting upload for file extracted_data.pkl
100%|█████████████████████████████████████████| 435k/435k [00:01<00:00, 334kB/s]
Upload successful: extracted_data.pkl (435KB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ishansrivastava1308/spotify-dataset


In [18]:
try:
    audio_output_paths  = scrapper.yt_scraper.threaded_download_song_by_video_id_list(track_data_df.video_id.to_numpy(), 'dataset/songs', max_workers = 50)
    track_data_df['audio_file_path'] = audio_output_paths
    scrapper.df['audio_file_path'] = audio_output_paths
    track_data_df.to_pickle('dataset/total_data.pkl')
    scrapper.df.to_pickle('dataset/extracted_data.pkl')
    !kaggle datasets version -m "Ishan 0 to 8400 audio done" -p /kaggle/working/dataset --dir-mode tar
except:
    audio_output_paths = scrapper.yt_scraper.threaded_download_song_by_video_id_list(scrapper.df.video_id.to_numpy(), 'dataset/songs', max_workers = 50)
    scrapper.df['audio_file_path'] = audio_output_paths
    !kaggle datasets version -m "Ishan 0 to 8400 salavge" -p /kaggle/working/dataset --dir-mode tar

100%|██████████| 200/200 [05:43<00:00,  1.72s/it]


{'success': 200, 'fail': 0}
Starting upload for file songs.tar
100%|████████████████████████████████████████| 134M/134M [00:05<00:00, 23.5MB/s]
Upload successful: songs.tar (134MB)
Starting upload for file extracted_data.pkl
100%|█████████████████████████████████████████| 435k/435k [00:02<00:00, 220kB/s]
Upload successful: extracted_data.pkl (435KB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ishansrivastava1308/spotify-dataset
